In [2]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [3]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [10]:
import json
from satori.rtm.client import make_client, SubscriptionMode
from math import log

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    n=99   # number of inputs
    numOfRendements=20
    """
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    lastPrice=float(json.loads(json.dumps(messagebox.pop(), ensure_ascii=False))['price']) # get that first price
    while True:
        proxies=np.empty(0) # x of the RLS algorithm
        while proxies.size<n+1:
            rendements=np.empty(0) 
            while rendements.size<numOfRendements: # Choix du nombre de rendements pour le calcul du proxy            
                if messagebox:
                    message=messagebox.pop()
                    message=json.dumps(message, ensure_ascii=False)
                    message=json.loads(message)
                    rendement=np.log(float(message['price'])/lastPrice) # r = np.log(float(message['price'])/lastPrice)
                    #print(float(message['price']), lastPrice, rendement)
                    lastPrice=float(message['price'])
                    rendements=np.append(rendements, rendement)
                    #print(float(message['price']), lastPrice, log(float(message['price'])/lastPrice))print(rendements, rendements.size)
                time.sleep(1)
            proxies=np.append(proxies, np.sum(np.square(rendements))/numOfRendements)
    """
    proxies=np.array([1.56431868e-06,  1.52459776e-06,
               9.92640792e-07,  4.79747330e-07,  1.63891281e-07,  7.88273988e-07,
               3.41413774e-07,  1.21605917e-06,  5.91032875e-07,  1.02962140e-06,
               7.29887874e-07,  2.37768278e-06,  1.95345806e-06,  1.37050245e-06,
               5.76437732e-07,  2.91808064e-07,  7.12492254e-07,  4.24287628e-07,
               1.47932748e-06,  1.10337251e-07,  1.46921057e-08,  4.92506056e-07,
               6.94714509e-07,  1.01885561e-06,  2.72151229e-06,  8.08214515e-07,
               1.39947311e-06,  1.06234534e-06,  1.13654781e-06,  9.51392622e-07,
               6.62724032e-07,  2.27373572e-07,  9.65890902e-08,  2.00190401e-07,
               7.83200744e-08,  1.01007445e-07,  9.38455764e-08,  2.56804218e-07,
               2.64894009e-07,  1.08449055e-06,  1.29404779e-07,  8.24094332e-07,
               9.32365418e-07,  1.72536500e-06,  9.13574384e-07,  7.63601813e-07,
               2.03124775e-06,  2.46166365e-07,  1.34944748e-06,  1.61951227e-06,
               1.28458063e-06,  1.40513451e-06,  1.69283264e-06,  1.54956918e-06,
               1.46694598e-06,  3.88917438e-06,  1.78506348e-06,  1.12225740e-06,
               2.31985321e-06,  5.80976554e-07,  8.51692132e-07,  2.84822748e-07,
               2.36251845e-07,  1.71578475e-06,  5.99307604e-07,  3.77918230e-07,
               1.68820076e-07,  1.11673647e-06,  2.43383484e-06,  1.47138731e-06,
               8.36762491e-07,  7.70235361e-07,  4.83113253e-07,  5.39407000e-07,
               6.62463640e-07,  1.64492108e-06,  1.31922336e-06,  6.74131836e-07,
               6.21040870e-07,  5.31915709e-07,  8.86563749e-07,  4.04694613e-07,
               5.28815201e-07,  6.70167845e-07,  5.31399982e-07,  5.81826692e-07,
               7.00886640e-07,  8.16382830e-07,  9.75436557e-07,  8.69988979e-08,
               1.09499659e-06,  7.31087622e-07,  9.75984606e-07,  1.64839493e-06,
               1.43895203e-06,  2.63635756e-06,  4.12855435e-07,  1.83715255e-06,
               2.70916759e-06,  1.04962679e-06])
    print('before normalization', proxies)
    proxies=(proxies-np.mean(proxies))/np.var(proxies)
    print('after normalization', proxies)
    y=proxies[-1]
    proxies=np.delete(proxies, proxies.size-1)
    message=np.array2string(np.append([i,y], proxies),separator=",") 
    producer.send('test2', message)
    i=i+1
    print(message)
    """
        while True:
            proxies=np.roll(proxies,-1)
            proxies[0]=1
            while not messagebox:
                time.sleep(1)
            message=messagebox.pop()
            message=json.dumps(message, ensure_ascii=False)
            message=json.loads(message)
            proxy+=(np.log(float(message['price'])/lastPrice)**2)/j # r = np.log(float(message['price'])/lastPrice)
            lastPrice=float(message['price'])
            proxies[n-1]=proxy
            j+=1
            y=np.sum(proxies)+0.1*np.random.rand(1)[0]
            message=np.array2string(np.append([i,y], proxies),separator=",") 
            producer.send('test2', message)
            i=i+1
            print(message)
    """

Connected to Satori RTM!
('before normalization', array([  1.56431868e-06,   1.52459776e-06,   9.92640792e-07,
         4.79747330e-07,   1.63891281e-07,   7.88273988e-07,
         3.41413774e-07,   1.21605917e-06,   5.91032875e-07,
         1.02962140e-06,   7.29887874e-07,   2.37768278e-06,
         1.95345806e-06,   1.37050245e-06,   5.76437732e-07,
         2.91808064e-07,   7.12492254e-07,   4.24287628e-07,
         1.47932748e-06,   1.10337251e-07,   1.46921057e-08,
         4.92506056e-07,   6.94714509e-07,   1.01885561e-06,
         2.72151229e-06,   8.08214515e-07,   1.39947311e-06,
         1.06234534e-06,   1.13654781e-06,   9.51392622e-07,
         6.62724032e-07,   2.27373572e-07,   9.65890902e-08,
         2.00190401e-07,   7.83200744e-08,   1.01007445e-07,
         9.38455764e-08,   2.56804218e-07,   2.64894009e-07,
         1.08449055e-06,   1.29404779e-07,   8.24094332e-07,
         9.32365418e-07,   1.72536500e-06,   9.13574384e-07,
         7.63601813e-07,   2.031247